<a href="https://colab.research.google.com/github/waishun78/hungry-rag/blob/main/hungry_rag.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# Installing the necessary packages
!pip install --upgrade pip
!pip install 'farm-haystack[all]' ## or 'all-gpu' for the GPU-enabled dependencies

!pip install -U accelerate
!pip install bitsandbytes
!pip install SentencePiece
#!pip install evaluate
#!pip install bert_score
!pip install transformers
#!pip install elasticsearch
#!pip install googlemaps
# !apt install libgraphviz-dev
# !pip install pygraphviz

  Obtaining dependency information for pip from https://files.pythonhosted.org/packages/47/6a/453160888fab7c6a432a6e25f8afe6256d0d9f2cbd25971021da6491d899/pip-23.3.1-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 9.9 MB/s eta 0:00:00:00:010:01
  Attempting uninstall: pip
    Found existing installation: pip 23.2.1
    Uninstalling pip-23.2.1:
      Successfully uninstalled pip-23.2.1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.7/10.7 MB 52.6 MB/s eta 0:00:0000:010:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.7/48.7 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.5/121.5 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.9/63.9 kB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 43.1 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
 

In [1]:
!nvidia-smi

Sat Dec  2 09:13:08 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.161.03   Driver Version: 470.161.03   CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla T4            Off  | 00000000:00:05.0 Off |                    0 |
| N/A   

In [4]:
# Import necessary modules
import os
from subprocess import Popen, PIPE, STDOUT
import torch
import torch.nn as nn
import bitsandbytes as bnb
import accelerate
from transformers import AutoTokenizer, AutoConfig, AutoModelForCausalLM
from transformers import LlamaTokenizer
from elasticsearch import helpers
from elasticsearch import Elasticsearch
import pandas as pd

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [5]:
load_in_8bit = True
model_name_l = "lmsys/vicuna-7b-v1.5"

model_l = AutoModelForCausalLM.from_pretrained(
    model_name_l,
    torch_dtype=torch.float16,
    load_in_8bit=load_in_8bit,
    device_map="auto"
)
tokenizer_l = LlamaTokenizer.from_pretrained(model_name_l)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:367: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(


In [6]:
from haystack.nodes.base import BaseComponent

class LLMAnswerGen(BaseComponent):
    outgoing_edges = 1

    def __init__(self, model, tokenizer):
        self.model = model
        self.tokenizer = tokenizer

    def run(self, query: str):

        prompt=f"""Generate Answer to the Questions according to the example below. Give 3 Options for Each Question.
                  Example:
                  Question: Where to get Japanese food in Marina Bay Sands that is fine-dining and has a romantic atmosphere at night?
                  Answer:
                  1. Sen of Japan is the best choice as it has serves sushi ('Lobster uni maki'), on top of that it has the highest rating of 4.3 out of the other options. Lastly, it was 'great for a date' as mentioned in the review. It is also open till 22:00 as needed.
                  2. A close scond would be KOMA Singapore. It has a great great interior lighting for a romantic atmospher and quality food. It also has a good rating of 4.3 but did not explicitly mention if sushi is served.
                  3. Waku Ghin is the third choice as it is more expensive than the other choices. It is 'totally not worth it even tho the food was great' as quoted from a review.
                  
                  Question: Where to get Indian food in Little India that is open late and has a view of the street?
                  Answer:
                  1. Khansama Tandoori Restaurant: This restaurant is located on Serangoon Road and is open until 11pm every night. They offer a wide variety of Indian dishes, including tandoori chicken, butter chicken, and biryani.
                  2. Banana Leaf Apolo Restaurant: Also located at 266 Race Course Road, Singapore 218217, this restaurant is open until 10 PM and offers a view of Serangoon Road.
                  
                  ____________________________________________________________________________
                  
                  Question: {query}
                  Answer:"""
        input = self.tokenizer(prompt, return_tensors='pt')
        input_ids = input["input_ids"].to("cuda")
        temperature=0.7
        with torch.no_grad():
            generation_output = self.model.generate(
                input_ids=input_ids,
                temperature=temperature,
                top_p = 1.0,
                do_sample=True,
                return_dict_in_generate=True,
                max_new_tokens=200,
              )

        s = generation_output.sequences[0][len(input_ids[0]):]

        output = self.tokenizer.decode(s)

        del s
        del input
        del input_ids
        torch.cuda.empty_cache()

        return {"query":query, "prompt":prompt, "output":output}, "output_1"

    def run_batch(self, queries):
        # Insert code here to manipulate the input and produce an output dictionary
        output = []
        for query in queries:
            output.append(self.run(query))
        return output, "output_1"

2023-12-02 09:20:10,770	INFO util.py:129 -- Outdated packages:
  ipywidgets==7.7.1 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


#  **Raw Answer Generation Pipeline (Pipeline #1)**

In [7]:
# Create Custom Answer Generator
answer_gen = LLMAnswerGen(model_l, tokenizer_l)

In [8]:
from haystack import Pipeline

# Create a Haystack pipeline
p = Pipeline()
p.add_node(component=answer_gen, name="LLM_Answer_Generator", inputs=["Query"])

In [9]:
mexican_query = "Where to get budget-friendly Mexican food in Orchard Road with vegan options?"

In [10]:
mexican_result = p.run(query = mexican_query)

In [11]:
print(f'''
  query: {mexican_result["query"]}
  output: {mexican_result["output"]}
  ''')


  query: Where to get budget-friendly Mexican food in Orchard Road with vegan options?
  output: 
                  1. Tacos Mariachi: This casual dining Mexican restaurant is located in the basement of Pacific Plaza mall and offers vegan options for their tacos and burritos. Prices are affordable and it is open until 22:00.
                  2. Guacamole's Mexican Grill & Margarita Bar: This casual dining Mexican restaurant is located in the basement of Pacific Plaza mall and offers vegan options for their tacos and burritos. Prices are affordable and it is open until 22:00.
                  3. Mexico Corner: This is a fast-food style Mexican restaurant located in the basement of Pacific Plaza mall. It is open until 21:00 and does have vegan options available. However, it may not be as budget-friendly as the other options.</s>
  


#  **Raw Answer Generation Pipeline with ES (Pipeline #2)**

In [12]:
!pip install elasticsearch

In [162]:
import time
class ESNode(BaseComponent):
    outgoing_edges = 1
    def __init__(self,hosturl):
        self.hosturl = hosturl

    def createServer(self):
        self.es_server = Popen(['elasticsearch-7.0.0/bin/elasticsearch'],
                  stdout=PIPE, stderr=STDOUT,
                  preexec_fn=lambda: os.setuid(1)  # as daemon
                 )

    def connectServer(self, settings):
        time.sleep(20)
        self.es = Elasticsearch(hosts = [{"host":"localhost", "port":9200}])
        if not self.es.indices.exists(index='food_index'):
            food_index = self.es.indices.create(index="food_index", ignore=[400,404], body=settings)
            print(food_index)
        print(f"The Elastic Server is active: {self.es.ping()}")

    def pingServer(self):
         print(f"The Elastic Server is active: {self.es.ping()}")

    def json_formatter(self, dataset, index_name, index_type='_doc'):
        """
        This function is used to create JSON formatted dictionaries for Elasticsearch.

        Args:
          dataset: The dataset you want to apply this function.
          index_name: Name of the index in Elasticsearch
          index_type: Type of the index in Elasticsearch.
          Note: It is suggested to keep index_type as '_doc' since it is deprecated from version 6.
          Note: This function formats all columns of your dataset, if you want to apply this to special columns only,
          you can delete the second for loop and add your custom fields.
        """
        try:
            List = []

            for k, v in dataset.items():
                for idx, row in enumerate(v):
                    dic = {}
                    dic['_index'] = index_name
                    #dic['_type'] = index_type
                    source = {}
                    for i in row.keys():
                        source[i] = row[i]
                    dic['_source'] = source
                    List.append(dic)
            return List

        except Exception as e:
            print("There is a problem: {}".format(e))

    def postData(self, json_Formatted_dataset):
        # For importing Data to elasticsearch we use elasticsearch's bulk API from elasticsearch.helpers
        try:
            res = helpers.bulk(self.es, json_Formatted_dataset)
            print("Successfully imported to elasticsearch.")
        except Exception as e:
            print(f"error: {e}")

    def runQuery(self, infoQuery):
        search = self.es.search(
            index="food_index",
            body={
                "size":3,
                "query":{
                    "bool":{
                        "must":[
                                {"match":{"query":infoQuery,
                                }}
                        ]
                    }
                }
            }
        )

        return search

    def formatResponse(self, searchResults):
        hits = searchResults['hits']['hits'][:3]
        final_context_text = ""
        for idx, hit in enumerate(hits):
            final_context_text += f'''\n
Option: {idx+1}:
Name: {hit['_source']['name']}
Address: {hit['_source']['address']}
Phone: {hit['_source']['phone_number']}
Delivery Available: {'Information Not Available' if hit['_source'].get('delivery') is None else 'Yes' if hit['_source']['delivery'] else 'No'}
Dine-In Available: {'Information Not Available' if hit['_source'].get('dine_in') is None else 'Yes' if hit['_source']['dine_in'] else 'No'}
Opening Hours:
{hit['_source']['opening_hours']}
Price Level: {hit['_source']['price_level']} / 4 (0- Free, 1- Inexpensive, 2- Moderate, 3- Expensive, 4- Very Expensive, None- Price Data Not Available)
Rating: {hit['_source']['rating']}
Reviews: {hit['_source']['reviews'][:200]}'''

        return final_context_text

    def run(self, query: dict):
        #query['context'] = ""
        print(f'Generating Context Response from ES index.... with query:{query}')
        search_terms = query["query"]
        print(f"Search Terms: {search_terms}\n\n\n")
        
        results = self.runQuery(search_terms)
        if len(results['hits']['hits']) == 0:
            raise Exception("No context")
        context= self.formatResponse(results)
        print(f'\n____CONTEXT:____\n {context} \n\n')
        return {"query":search_terms, "context":context},  "output_1"

    def run_batch(self, queries: [dict]):
        # Insert code here to manipulate the input and produce an output dictionary
        output = []
        for query in queries:
            output.append(self.run(query))
        return output, "output_1"

In [136]:
settings = {
    "settings":{
        "number_of_shards":1,
        "number_of_replicas":0
    },
    "mappings":{
        "properties":{
            "query":{
                "type":"text"
            },
            "name":{
                "type":"text"
            },
            "address":{
                "type":"text"
            },
            "opening_hours":{
                "type":"text"
            },
            "phone_number":{
                "type":"text"
            },
            "price_level":{
                "type":"integer"
            },
            "reviews":{
                "type":"text"
            },
            "delivery":{
                "type":"boolean"
            },
            "rating":{
                "type":"float"
            },
            "dine_in":{
                "type":"boolean"
            }
        }
    }
}

In [31]:
# Download & extract Elasticsearch 7.0.0

!wget https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-7.0.0-linux-x86_64.tar.gz -q
!tar -xzf elasticsearch-7.0.0-linux-x86_64.tar.gz
!chown -R daemon:daemon elasticsearch-7.0.0

In [163]:
#Instantiate an ES Server
haystack_elastic = ESNode("localhost:9200/")
haystack_elastic.createServer()

In [164]:
%%bash
# If you get 1 root & 2 daemon process then Elasticsearch instance has started successfully
ps -ef | grep elasticsearch

daemon       982      47  4 09:49 ?        00:01:11 /kaggle/working/elasticsearch-7.0.0/jdk/bin/java -Xms1g -Xmx1g -XX:+UseConcMarkSweepGC -XX:CMSInitiatingOccupancyFraction=75 -XX:+UseCMSInitiatingOccupancyOnly -Des.networkaddress.cache.ttl=60 -Des.networkaddress.cache.negative.ttl=10 -XX:+AlwaysPreTouch -Xss1m -Djava.awt.headless=true -Dfile.encoding=UTF-8 -Djna.nosys=true -XX:-OmitStackTraceInFastThrow -Dio.netty.noUnsafe=true -Dio.netty.noKeySetOptimization=true -Dio.netty.recycler.maxCapacityPerThread=0 -Dlog4j.shutdownHookEnabled=false -Dlog4j2.disable.jmx=true -Djava.io.tmpdir=/tmp/elasticsearch-17296973692464245136 -XX:+HeapDumpOnOutOfMemoryError -XX:HeapDumpPath=data -XX:ErrorFile=logs/hs_err_pid%p.log -Xlog:gc*,gc+age=trace,safepoint:file=logs/gc.log:utctime,pid,tags:filecount=32,filesize=64m -Djava.locale.providers=COMPAT -Dio.netty.allocator.type=unpooled -Des.path.home=/kaggle/working/elasticsearch-7.0.0 -Des.path.conf=/kaggle/working/elasticsearch-7.0.0/config -Des.distri

In [165]:
#connect to instantiated daemon
haystack_elastic.connectServer(settings)

The Elastic Server is active: True


In [166]:
import json
dataset = json.load(open("/kaggle/input/google-maps-scraped-data/FB Dataset.json", "r"))
json_Formatted_dataset = haystack_elastic.json_formatter(dataset=dataset, index_name='food_index')
haystack_elastic.postData(json_Formatted_dataset)

/tmp/ipykernel_47/1731088888.py:2: ResourceWarning: unclosed file <_io.TextIOWrapper name='/kaggle/input/google-maps-scraped-data/FB Dataset.json' mode='r' encoding='ANSI_X3.4-1968'>
  dataset = json.load(open("/kaggle/input/google-maps-scraped-data/FB Dataset.json", "r"))


Successfully imported to elasticsearch.


In [167]:
from haystack.nodes.base import BaseComponent

class ESAnswerGen(BaseComponent):
    outgoing_edges = 1

    def __init__(self, model, tokenizer):
        self.model = model
        self.tokenizer = tokenizer

    def run(self, query: dict):
        print(f'Generating Response from LLM.... with query:{query}')
        prompt=f"""Generate Answer to the Questions according to the example below.
                  Example:
                  Question: Where to get Japanese food in Marina Bay Sands that is fine-dining and has a romantic atmosphere at night?
                  Context:
                        Option: 1:
                        Name: Waku Ghin
                        Address: Level 2 Dining, L2-03 The Shoppes at, 2 Bayfront Ave, Marina Bay Sands, 018956


                        Price Level: 4 / 4 (0- Free, 1- Inexpensive, 2- Moderate, 3- Expensive, 4- Very Expensive, None- Price Data Not Available)
                        Rating: 4.1
                        Review 1: Absolutely amazing food; had the Omakase at the chef’s table. The umami and creamy sea urchin balanced with the sweetness of the shrimp and slight saltiness of the caviar to produce a full flavour profile with each mouthful. Loved the perfectly (teppanyaki) seared abalone with risoni and the A5 Kobe wagyu dipped in the egg yolk (it was so silky, tender and flavourful from the marbling). Service was excellent as well; they were jovial and attentive and made the experience delightful.
                        Review 2: This is a late review. I came here during valentines weekend. Both my friend and I had food poisoning and I was bed ridden for three days. Only decided to review now because my friend told me that her group of friends also got food poisoning from the same group of restaurants. So totally not worth it even tho the food was great. Spent 750 per pax to get food poisoning.. really ridiculous..
                        Review 3: We had some Izakaya food and beer before going for show. Food must be good considering the price is not cheap for beer food, but its a tad a bit too salty for me. Service is great of course. My favorite will be the lobster spaghetti, a bit of fusion.

                        Option: 2:
                        Name: Sen of Japan
                        Address: 2 Bayfront Ave, #01 - 86, Singapore 018972

                        Price Level: 3 / 4 (0- Free, 1- Inexpensive, 2- Moderate, 3- Expensive, 4- Very Expensive, None- Price Data Not Available)
                        Rating: 4.4
                        Review 1: Lunch with a view.

                        The marugo pizza is really nice and thin.

                        Pork jowl is thinly slice. The thinnest I’ve seen. Nice in paring with the sauce

                        Black Cod fish is soft and nicely cook too.

                        V worth it if you have American Express card , to enjoy 50% disc on the bill. This meal of 3 fish cause only $80+.

                        They also have some pretty affordable lunch set.
                        Review 2: Five stars for everything (except maybe the receptionist that received us.. kind of rude). The chefs were super friendly (sat at the counter), and I had nice small talk with them while eating.
                        Review 3: Excellent upscale Japanese restaurant great for a date.
                        Lobster uni maki was out of this world. Amazing flavours and textures in a single mouthful.

                        Option: 3:
                        Name: KOMA Singapore
                        Address: 2 Bayfront Ave, # B1 - 67, Singapore 018972

                        Price Level: None / 4 (0- Free, 1- Inexpensive, 2- Moderate, 3- Expensive, 4- Very Expensive, None- Price Data Not Available)
                        Rating: 4.3
                        Review 1: Really nice restaurant with great food and perfect service. The atmosphere can get quite uncomfortable because it is always packed. Prices are reasonable for quality and location. Very recommended for date nights.
                        Review 2: Obviously, the main attraction would be the interior lighting. Can't deny the entire place makes a great backdrop for IG-worthy photos.
                        Review 3: Celebrated my wife birthday there recently, was a very nice place. Food was truly amazing and the staffs were very friendly and helpful in many ways.
                  Answer:
                  1. Sen of Japan is the best choice as it has serves sushi ('Lobster uni maki'), on top of that it has the highest rating of 4.3 out of the other options. Lastly, it was 'great for a date' as mentioned in the review. It is also open till 22:00 as needed.
                  2. A close scond would be KOMA Singapore. It has a great great interior lighting for a romantic atmospher and quality food. It also has a good rating of 4.3 but did not explicitly mention if sushi is served.
                  3. Waku Ghin is the third choice as it is more expensive than the other choices. It is 'totally not worth it even tho the food was great' as quoted from a review.
                  ____________________________________________________________________________
                  
                  Question: {query["query"]}
                  Context:
                  {query["context"]}
                  Answer:"""
        input = self.tokenizer(prompt, return_tensors='pt')
        input_ids = input["input_ids"].to("cuda")
        temperature=0.7
        with torch.no_grad():
            generation_output = self.model.generate(
                input_ids=input_ids,
                temperature=temperature,
                top_p = 1.0,
                do_sample=True,
                return_dict_in_generate=True,
                max_new_tokens=200,
              )

        s = generation_output.sequences[0][len(input_ids[0]):]

        output = self.tokenizer.decode(s)

        del s
        del input
        del input_ids
        torch.cuda.empty_cache()
        
        query["output"] = output

        return query, "output_1"

    def run_batch(self, queries):
        # Insert code here to manipulate the input and produce an output dictionary
        output = []
        for query in queries:
            output.append(self.run(query))
        return output, "output_1"

In [168]:
es_llm = ESAnswerGen(model_l, tokenizer_l)

In [169]:
# Create a Haystack pipeline
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

p2 = Pipeline()
p2.add_node(component=haystack_elastic, name="ES_Index", inputs=["Query"])
#p2.add_node(component=es_llm, name="ES_LLM", inputs=["ES_Index"])

In [170]:
mexican_query = {"query":"Where to get budget-friendly Mexican food in Orchard Road with vegan options?"}

In [171]:
mexican_result = p2.run(query = mexican_query)

Generating Context Response from ES index.... with query:{'query': 'Where to get budget-friendly Mexican food in Orchard Road with vegan options?'}
Search Terms: Where to get budget-friendly Mexican food in Orchard Road with vegan options?




____CONTEXT:____
 

Option: 1:
Name: Joie | Vegetarian Casual Fine Dining
Address: 181 Orchard Rd, #12 - 01, Singapore 238896
Phone: 9663 0901
Delivery Available: Information Not Available
Dine-In Available: Yes
Opening Hours:
Monday: 12:00 – 3:00 PM, 6:00 – 10:00 PM
Tuesday: 12:00 – 3:00 PM, 6:00 – 10:00 PM
Wednesday: 12:00 – 3:00 PM, 6:00 – 10:00 PM
Thursday: 12:00 – 3:00 PM, 6:00 – 10:00 PM
Friday: 12:00 – 3:00 PM, 6:00 – 10:00 PM
Saturday: 12:00 – 3:00 PM, 6:00 – 10:00 PM
Sunday: 12:00 – 3:00 PM, 6:00 – 10:00 PM

Price Level: 3 / 4 (0- Free, 1- Inexpensive, 2- Moderate, 3- Expensive, 4- Very Expensive, None- Price Data Not Available)
Rating: 4.5
Reviews: 

Nice vegan and vegetarian option. But the vegan food was lacking in flavor. The atmosph

In [172]:
final_res = es_llm.run(mexican_result)

Generating Response from LLM.... with query:{'query': 'Where to get budget-friendly Mexican food in Orchard Road with vegan options?', 'context': "\n\nOption: 1:\nName: Joie | Vegetarian Casual Fine Dining\nAddress: 181 Orchard Rd, #12 - 01, Singapore 238896\nPhone: 9663 0901\nDelivery Available: Information Not Available\nDine-In Available: Yes\nOpening Hours:\nMonday: 12:00\u2009–\u20093:00\u202fPM, 6:00\u2009–\u200910:00\u202fPM\nTuesday: 12:00\u2009–\u20093:00\u202fPM, 6:00\u2009–\u200910:00\u202fPM\nWednesday: 12:00\u2009–\u20093:00\u202fPM, 6:00\u2009–\u200910:00\u202fPM\nThursday: 12:00\u2009–\u20093:00\u202fPM, 6:00\u2009–\u200910:00\u202fPM\nFriday: 12:00\u2009–\u20093:00\u202fPM, 6:00\u2009–\u200910:00\u202fPM\nSaturday: 12:00\u2009–\u20093:00\u202fPM, 6:00\u2009–\u200910:00\u202fPM\nSunday: 12:00\u2009–\u20093:00\u202fPM, 6:00\u2009–\u200910:00\u202fPM\n\nPrice Level: 3 / 4 (0- Free, 1- Inexpensive, 2- Moderate, 3- Expensive, 4- Very Expensive, None- Price Data Not Available

In [173]:
print(f'''
  output: {str(final_res[0]['output'])}
  ''')


  output: 
                  1. Joie | Vegetarian Casual Fine Dining is the best choice as it has the highest rating of 4.5 out of the other options, and it has vegan options. It is also open till late as needed.
                  2. Original Sin Restaurant is the second choice as it has a good rating of 4.3 and it is open for dine-in. It also has vegetarian options.
                  3. Genesis Vegan Restaurant is the third choice as it is the most affordable option out of the other options, it has a great rating of 4.4 and it is open for dine-in.
                  ____________________________________________________________________________
                  
                  Question: Where to get a high-quality sushi in the Marina Bay Sands area at a reasonable price?
                  Context:
                  
                  Option: 1
  
